In [1]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import year, month
from pyspark.sql.functions import to_timestamp,date_format

spark = SparkSession.builder.getOrCreate()

In [2]:
df1 = spark.read.load('/mnt/prepared/SupplyChain/StoreOrder/StoreOrderLine')

df2 = spark.read.load('dbfs:/mnt/thorcf2/clean/supp/dedi/storeorder')

df2 = (
    df2.select("Header.*", "Payload.*", "DeliveryDatePartitionKey", "__meta.*")
    .select("Event.*", "StoreOrder.*", "DeliveryDatePartitionKey", "ModifiedTimestamp")
    .select("Action", "DateTime", "Opco.*", "StoreNo", "WarehouseNo","StoreOrderNo","MessageCreationDatetime","DeliveryStreamNo","DeliveryDateTime",
            "DelivererNo","SupplyMethod","StoreOrderLine","DeliveryDatePartitionKey","ModifiedTimestamp")
    .select("Action","DateTime","CountryCode","GLN","StoreNo","WarehouseNo","StoreOrderNo","MessageCreationDatetime",
            "DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod",*[F.col('StoreOrderLine')[i].alias(f'StoreOrderLine{i}') for i in range(1)],
            "DeliveryDatePartitionKey","ModifiedTimestamp")
    .select("Action","DateTime","CountryCode","GLN","StoreNo","WarehouseNo","StoreOrderNo","MessageCreationDatetime","DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod","StoreOrderLine0.*","DeliveryDatePartitionKey","ModifiedTimestamp")
)
df2 = df2.withColumnRenamed('SoiNo','Soino')
df2 = df2.withColumn("date", to_date(col("DeliveryDateTime")))
df2 = df2.withColumn('year', year(df2.date))
df2 = df2.withColumn('month', month(df2.date))
df2 = df2.withColumn("date",to_timestamp(col("date"))).withColumn("day", date_format(col("date"), "d"))

df2 = df2.withColumn("date", to_date(col("DeliveryDateTime")))
df2 = df2.withColumn('year', year(df2.date))
df2 = df2.withColumn('month', month(df2.date))
df2 = df2.withColumn("date",to_timestamp(col("date"))).withColumn("day", date_format(col("date"), "d"))

df2.printSchema()

root
 |-- Action: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- GLN: long (nullable = true)
 |-- StoreNo: integer (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- StoreOrderNo: integer (nullable = true)
 |-- MessageCreationDatetime: timestamp (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- DelivererNo: integer (nullable = true)
 |-- SupplyMethod: string (nullable = true)
 |-- Soino: long (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- OrderLineId: integer (nullable = true)
 |-- PromotionInd: string (nullable = true)
 |-- InitialPromotionInd: string (nullable = true)
 |-- SupplierCompanyGln: long (nullable = true)
 |-- SupplierNo: integer (nullable = true)
 |-- AddUnorderedInd: string (nullable = true)
 |-- EmergencyOrderInd: string (nullable = true)
 |-- DeliveryDatePartitionKey: integer (nullable = true)
 |--

# Data validatie

In [4]:
df1_copy = df1
df2_copy = df2
df1_copy = df1_copy.filter(df1_copy.year > 2021)

df1_copy = df1_copy.drop(
    'StoreNo',
    'StoreOrderNo',
)

# df1_copy.printSchema()
# df2_copy.printSchema()

In [5]:
df2_copy = df2_copy.withColumnRenamed('DateTime','InsertDate')
df2_copy = df2_copy.drop(
    'Action',
    'CountryCode',
    'GLN','StoreNo',
    'MessageCreationDatetime',
    'StoreOrderNo',
    'ModifiedTimestamp',
    'DeliveryDatePartitionKey',
    'DelivererNo',
    'SupplyMethod'   
)

df1_copy.printSchema()
df2_copy.printSchema()

root
 |-- InsertDate: timestamp (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- Soino: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- OrderLineId: integer (nullable = true)
 |-- PromotionInd: string (nullable = true)
 |-- InitialPromotionInd: string (nullable = true)
 |-- SupplierCompanyGln: string (nullable = true)
 |-- SupplierNo: integer (nullable = true)
 |-- AddUnorderedInd: string (nullable = true)
 |-- EmergencyOrderInd: string (nullable = true)
 |-- date: date (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)

root
 |-- InsertDate: timestamp (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- Soino: long (nullable = true)
 |-- Quantity: integer (nul

In [5]:
# df1_copy.select('StoreOrderNo').show()
# df2_copy.select('DelivererNo').show()

# diff = df1_copy.select('StoreOrderNo').subtract(df2_copy.select('DelivererNo'))
# diff2 = df2_copy.select('DelivererNo').subtract(df1_copy.select('StoreOrderNo'))
# diff2.count()

# print(diff.count())
# df1_copy.count()

In [6]:
# df_difference = df1_copy.subtract(df2_copy)
# df_difference.show()
# df_difference.count()
# df2 is substracted van df1 om de resultaten die in df1 zitten, maar niet in df2 zitten te tonen. 1372594136 records komen dus niet overeen. 6.677.544 records 

In [7]:
# df_difference2 = df2_copy.subtract(df1_copy)
# df_difference2.show()
# df_difference2.count()

In [8]:
# print(f'df1_copy: count: {df1_copy.count()}')
# print(f'df2_copy: count: {df2_copy.count()}')

5.723.649 van de 6.260.161 van de rijen in df2 komen niet overeen met df1

In [6]:
result = df1_copy.join(df2_copy, on=df1_copy.columns, how="inner")
result.show()

+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|         InsertDate|WarehouseNo|DeliveryStreamNo|   DeliveryDateTime| Soino|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|      date|year|month|day|
+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|2022-03-10 18:37:40|        900|              96|2022-03-11 10:30:00|151700|       2|          1|           N|                  N|     8712423008113|      5462|              N|                N|2022-03-11|2022|   03| 11|
|2022-04-09 15:13:52|         99|               4|2022-04-11 12:00:00|  2565|       1|          1|           N| 

In [7]:
df_joined = df2_copy.join(df1_copy, on=df1_copy.columns, how='inner') 
df_joined.show()

+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+-------------------+----+-----+---+
|         InsertDate|WarehouseNo|DeliveryStreamNo|   DeliveryDateTime| Soino|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|               date|year|month|day|
+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+-------------------+----+-----+---+
|2022-03-10 18:37:40|        900|              96|2022-03-11 10:30:00|151700|       2|          1|           N|                  N|     8712423008113|      5462|              N|                N|2022-03-11 00:00:00|2022|    3| 11|
|2022-04-09 15:13:52|         99|               4|2022-04-11 12:00:00|  2565

In [9]:
df_joined.count()

498

In [26]:
df1_316 = df1_copy.filter((df1_copy.OrderLineId == 316) & (df1_copy.SupplierNo == 5462))
df2_316 = df2_copy.filter((df2_copy.OrderLineId == 316) & (df2_copy.SupplierNo == 5462))

In [33]:
df1_316.show()

+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|         InsertDate|WarehouseNo|DeliveryStreamNo|   DeliveryDateTime| Soino|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|      date|year|month|day|
+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|2022-12-22 22:45:24|        900|              96|2022-12-23 19:30:00|151700|       4|        316|           N|                  N|     8712423008113|      5462|              N|                N|2022-12-23|2022|   12| 23|
|2022-12-22 22:00:29|        900|              96|2022-12-23 14:00:00|505562|       1|        316|           N| 

In [27]:
print(df1_316.count())
print(df2_316.count())

3409
0


In [35]:
df2_copy.filter((df2_copy['SupplierNo'] == 5462)&(df2_copy['OrderLineId'] == 316)).show()

+----------+-----------+----------------+----------------+-----+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----+----+-----+---+
|InsertDate|WarehouseNo|DeliveryStreamNo|DeliveryDateTime|Soino|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|date|year|month|day|
+----------+-----------+----------------+----------------+-----+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----+----+-----+---+
+----------+-----------+----------------+----------------+-----+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----+----+-----+---+



In [36]:
df_joined2 = df1_copy.join(df2_copy, on=df1_copy.columns, how='inner') 
df_joined2.show()
df_joined2.count()

+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|         InsertDate|WarehouseNo|DeliveryStreamNo|   DeliveryDateTime| Soino|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|      date|year|month|day|
+-------------------+-----------+----------------+-------------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|2022-03-10 18:37:40|        900|              96|2022-03-11 10:30:00|151700|       2|          1|           N|                  N|     8712423008113|      5462|              N|                N|2022-03-11|2022|   03| 11|
|2022-04-09 15:13:52|         99|               4|2022-04-11 12:00:00|  2565|       1|          1|           N| 

498